In [43]:
'''
TORCH.AUTOGRAD를 사용한 자동미분

신경망을 학습할 때 가장 주주 사용되는 알고리즘은 역전파입니다. 이 알고리즘에서 매개변수(모델가중치)는 주어진 매개변수에 대한 손실 함수(gradient)의 
변화도에 따라 조정됩니다.

이러한 변화도를 계산하기 위해 PyTorch에는 torch.autograd라고 불리는 자동 미분 엔진이 내장되어 있습니다. 이는 모든 게산 그래프에 대한 변화도의
자동 계산을 지원합니다.

입력 x, 매개변수 w와 b, 그리고 일부 손실함수가 있는 가장 간단한 단일 계층 신경망을 가정하겠습니다. PyTorch에서는 다음과 같이 정의할 수 잇습니다.
'''

'\nTORCH.AUTOGRAD를 사용한 자동미분\n\n신경망을 학습할 때 가장 주주 사용되는 알고리즘은 역전파입니다. 이 알고리즘에서 매개변수(모델가중치)는 주어진 매개변수에 대한 손실 함수(gradient)의 \n변화도에 따라 조정됩니다.\n\n이러한 변화도를 계산하기 위해 PyTorch에는 torch.autograd라고 불리는 자동 미분 엔진이 내장되어 있습니다. 이는 모든 게산 그래프에 대한 변화도의\n자동 계산을 지원합니다.\n\n입력 x, 매개변수 w와 b, 그리고 일부 손실함수가 있는 가장 간단한 단일 계층 신경망을 가정하겠습니다. PyTorch에서는 다음과 같이 정의할 수 잇습니다.\n'

In [44]:
import torch
x=torch.ones(5)
y=torch.zeros(3)
w=torch.randn(5,3,requires_grad=True)
b=torch.randn(3,requires_grad=True) #https://wikidocs.net/156999 requires_grad의 속성은 gradient(순간변화도,기울기)의 변화를 보기위해서 지정해주는 속성이다.
z=torch.matmul(x,w)+b
loss=torch.nn.functional.binary_cross_entropy_with_logits(z,y)

In [45]:
'''
이 신경망에서 w와 b는 최적화를 해야 하는 매개변수입니다. 따라서 이러한 변수들에 대한 손실 함수(loss function or cost function)의 변화도를 계산할 수 있어야 합니다.
이를 위해서 해당 텐서에 requires_grad속성을 설정합니다.
'''

'''
requires_grad의 값은 텐서를 생성할 때 설정하거나, 나중에 x.requires_grad(True)메소드를 사용하여 나중에 설정할 수도 있습니다.
'''

'\nrequires_grad의 값은 텐서를 생성할 때 설정하거나, 나중에 x.requires_grad(True)메소드를 사용하여 나중에 설정할 수도 있습니다.\n'

In [46]:
'''
연산 그래프를 구성하기 위해 텐서에 적용하는 함수는 사실 function클래스의 객체입니다. 이 객체는 순전파 방향으로 함수를 계산하는 방법과, 역전파단계에서
도함수(derivative)를 계산하는 방법을 알고 있습니다. 역방향 전파 함수에 대한 참조(reference)는 텐서의 grad_fn속성에 저장됩니다.
'''

'\n연산 그래프를 구성하기 위해 텐서에 적용하는 함수는 사실 function클래스의 객체입니다. 이 객체는 순전파 방향으로 함수를 계산하는 방법과, 역전파단계에서\n도함수(derivative)를 계산하는 방법을 알고 있습니다. 역방향 전파 함수에 대한 참조(reference)는 텐서의 grad_fn속성에 저장됩니다.\n'

In [47]:
print("Gradient Function for z=",z.grad_fn)
print("Gradient Function for loss=",loss.grad_fn)

Gradient Function for z= <AddBackward0 object at 0x00000290E6FEBA00>
Gradient Function for loss= <BinaryCrossEntropyWithLogitsBackward0 object at 0x00000290E6FEB2E0>


In [48]:
'''
변화도(Gradient)계산하기

신경망에서 매개변수의 가중치를 최적화하려면 매개변수에 대한 손실함수의 도함수(derivative)를 계산해야합니다. 즉, x와y의 일부 고정값에서 ∂loss/∂w 와 ∂loss/∂b가
필요합니다. 이러한 도함수를 계산하기 위해, loss.backward()를 호출한 다음 w.grad와 b.grad에서 값을 가져옵니다.
'''



'\n변화도(Gradient)계산하기\n\n신경망에서 매개변수의 가중치를 최적화하려면 매개변수에 대한 손실함수의 도함수(derivative)를 계산해야합니다. 즉, x와y의 일부 고정값에서 ∂loss/∂w 와 ∂loss/∂b가\n필요합니다. 이러한 도함수를 계산하기 위해, loss.backward()를 호출한 다음 w.grad와 b.grad에서 값을 가져옵니다.\n'

In [49]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1007, 0.0562, 0.0521],
        [0.1007, 0.0562, 0.0521],
        [0.1007, 0.0562, 0.0521],
        [0.1007, 0.0562, 0.0521],
        [0.1007, 0.0562, 0.0521]])
tensor([0.1007, 0.0562, 0.0521])


In [50]:
'''
1. 연산그래프의 leaf노드들 중 requires_grad속성이 True로 설정된 노드들의 grad속성만 구할 수 있습니다. 그래프의 다른 모든 노드에서는 변화도가 유효하지 않습니다.
2. 성능 상의 이유로, 주어진 그래프에서의 backward를 사용한 변화도 계산은 한 번만 수행할 수 있습니다. 만약 동일한 그래프에서 여러번의 backward호출이 필요하면, 
    backward호출 시에 retrain_graph=True를 전달해야 합니다.
'''

'\n1. 연산그래프의 leaf노드들 중 requires_grad속성이 True로 설정된 노드들의 grad속성만 구할 수 있습니다. 그래프의 다른 모든 노드에서는 변화도가 유효하지 않습니다.\n2. 성능 상의 이유로, 주어진 그래프에서의 backward를 사용한 변화도 계산은 한 번만 수행할 수 있습니다. 만약 동일한 그래프에서 여러번의 backward호출이 필요하면, \n    backward호출 시에 retrain_graph=True를 전달해야 합니다.\n'

In [51]:
'''
변화도 추적 멈추기

기본적으로, requires_grad=True인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원합니다. 그러나 모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우와
같이 순전파연산만 필요하 경우에는, 이러한 추적이나 지원이 필요없을 수 있습니다. 연산코드를 torch.no_grad()블록으로 둘러싸서 연산 추적을 멈출 수 있습니다.
'''

'\n변화도 추적 멈추기\n\n기본적으로, requires_grad=True인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원합니다. 그러나 모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우와\n같이 순전파연산만 필요하 경우에는, 이러한 추적이나 지원이 필요없을 수 있습니다. 연산코드를 torch.no_grad()블록으로 둘러싸서 연산 추적을 멈출 수 있습니다.\n'

In [53]:
z=torch.matmul(x,w)+b
print(z.requires_grad)

with torch.no_grad():
    z=torch.matmul(x,w)+b
print(z.requires_grad)

True
False


In [54]:
'''
동일한 결과를 얻는 다른 방법은텐서에 detach()메소드를 사용하는 것입니다.
'''
z=torch.matmul(x,w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


In [55]:
'''
변화도 추적을 멈춰야 하는 이유들은 다음과 같습니다.

1. 신경망의 일부 매개변수를 고정된 매개변수(frozen parameter)로 표시합니다. 이는 사전 학습된 신경망을 미세조정 할 때 매우 일반적인 시나리오입니다.
2. 변화도를 추적하지 않는 텐서의 연산이 더 효율적이기 때문에, 순전파 단계만 수행할 때 연산속도가 향상됩니다.
'''

'\n변화도 추적을 멈춰야 하는 이유들은 다음과 같습니다.\n\n1. 신경망의 일부 매개변수를 고정된 매개변수(frozen parameter)로 표시합니다. 이는 사전 학습된 신경망을 미세조정 할 때 매우 일반적인 시나리오입니다.\n2. 변화도를 추적하지 않는 텐서의 연산이 더 효율적이기 때문에, 순전파 단계만 수행할 때 연산속도가 향상됩니다.\n'

In [56]:
'''
연산그래프에 대한 추가 정보

개념적으로,autograd는 데이터(tensor) 및 실행된 모든 연산들(및 연산 결과가 새로운 텐서인 경우도 포함하여)의 기록을 Function객체로 구성된 
방향성 비순환 그래프(DAG; Directed Acyclic Graph)에 저장합니다. 이 방향성 비순환 그래프(DAG)의 leaf은 입력덴서이고, root는 결과 텐서입니다.
이 그래프를 root에서부터 leaf까지 추적하면 연쇄법칙(chain rule)에 따라 변화도를 자동으로 계산할 수 있습니다.
'''

'\n연산그래프에 대한 추가 정보\n\n개념적으로,autograd는 데이터(tensor) 및 실행된 모든 연산들(및 연산 결과가 새로운 텐서인 경우도 포함하여)의 기록을 Function객체로 구성된 \n방향성 비순환 그래프(DAG; Directed Acyclic Graph)에 저장합니다. 이 방향성 비순환 그래프(DAG)의 leaf은 입력덴서이고, root는 결과 텐서입니다.\n이 그래프를 root에서부터 leaf까지 추적하면 연쇄법칙(chain rule)에 따라 변화도를 자동으로 계산할 수 있습니다.\n'